# Childrens Act Disposals

In [78]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re
import os  # for file paths
import awswrangler as wr

In [79]:
# Structured  dataframes
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [80]:
# Assigning databases and S3 bucket
db1 = "familyman_dev_v2"
fcsq_db = "fcsq"
# create path for within athena FCSQ database in the S3 folder
fcsq_db_path = f"s3://alpha-family-data/fcsq_processing/CA_disps/"
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")
snapshot_date = "2021-11-11"

### Orders made

In [82]:
pydb.create_temp_table( 
f"""
  SELECT 
    DISTINCT
    case_number,
    disp_date,
    Year,
    Quarter,
    case_type,
    receipt_date,
    entry_date,
    event,
    event_model,
    field_model,
    order_type,
    disp_type_code,
    disp_type,
    order_code,
    order_desc,
    creating_court
  FROM 
    {fcsq_db}.ca_disps a 
  WHERE 
    disp_type_code = 1
""",

"ca_disp_order_type")

In [13]:
pydb.read_sql_query ("select count (*) count from __temp__.ca_order_type where year = 2020 and case_type = 'P' and order_code = 29 ")

,count
0,46368


### Disposal Events

In [83]:
pydb.create_temp_table( 
f"""
  SELECT 
    DISTINCT
    case_number,
    disp_date,
    Year,
    Quarter,
    case_type,
    receipt_date,
    entry_date,
    event,
    event_model,
    field_model,
    order_type,
    disp_type_code,
    disp_type,
    order_code,
    order_desc,
    creating_court,
    FO_value
  FROM 
    {fcsq_db}.ca_disps a 
  WHERE 
    order_code < 40
""",

"ca_disp_events")

In [21]:
pydb.read_sql_query ("select count (*) count from __temp__.ca_disp_events where year = 2011 and case_type = 'P'")

,count
0,122138


### Create flag for earliest/latest final order, non final order date - this can be moved to earier (or final order extraction can be moved to here)

In [84]:
pydb.create_temp_table(
f"""
Select
    *,
    ROW_NUMBER() OVER(PARTITION BY case_number, fo_value
                       ORDER BY case_number, disp_date ASC) 
      AS min_date,
    ROW_NUMBER() OVER(PARTITION BY case_number, fo_value
                       ORDER BY case_number, disp_date DESC) 
      AS max_date  
FROM
    __temp__.ca_disp_events
""",

"ca_FO_dates")
 
    

### Extract case closed events and create flag for earliest/latest event

In [24]:
pydb.create_temp_table(
f"""
Select
    case_number,
    receipt_date,
    entry_date,
    ROW_NUMBER() OVER(PARTITION BY case_number
                       ORDER BY case_number, entry_date ASC) 
      AS min_date,
    ROW_NUMBER() OVER(PARTITION BY case_number
                       ORDER BY case_number, entry_date DESC) 
      AS max_date  
FROM
    {db1}.events
WHERE
  event_model = 'U24'
  AND error = 'N'
  AND mojap_snapshot_date = date '{snapshot_date}'
""",

"ca_case_closed_events")
 

### Extract case reopened events and create flag for earliest/latest event

In [53]:
pydb.create_temp_table(
f"""
Select
    case_number,
    receipt_date,
    entry_date,
    ROW_NUMBER() OVER(PARTITION BY case_number
                       ORDER BY case_number, entry_date ASC) 
      AS min_date,
    ROW_NUMBER() OVER(PARTITION BY case_number
                       ORDER BY case_number, entry_date DESC) 
      AS max_date  
FROM
    {db1}.events
WHERE
  event_model = 'G62'
  AND error = 'N'
  AND mojap_snapshot_date = date '{snapshot_date}'
""",

"ca_case_reopened_events")
 

In [33]:
pydb.read_sql_query ("select * from __temp__.ca_case_reopened_events")

,case_number,receipt_date,entry_date,min_date,max_date
0,MF08P00153,NaT,2009-09-03 13:35:50,4,1
1,MF08P00153,NaT,2009-06-11 09:55:06,3,2
2,MF08P00153,NaT,2009-01-09 11:32:14,2,3
3,MF08P00153,NaT,2008-11-20 12:13:48,1,4


In [86]:
pydb.create_temp_table(
f"""
Select
    t1.case_number,
    (SELECT f.entry_date FROM __temp__.ca_FO_dates f
                         WHERE f.min_date = 1
                         AND f.fo_value = 'Y'
                         AND t1.case_number = f.case_number)
        AS first_final_order,
    (SELECT f.entry_date FROM __temp__.ca_FO_dates f
                         WHERE f.max_date = 1 
                         AND f.fo_value = 'Y'
                         AND t1.case_number = f.case_number)
        AS last_final_order,
    (SELECT f.entry_date FROM __temp__.ca_FO_dates f
                         WHERE f.min_date = 1
                         AND f.fo_value = 'N'
                         AND t1.case_number = f.case_number)
        AS first_no_final_order,
    (SELECT f.entry_date FROM __temp__.ca_FO_dates f
                         WHERE f.max_date = 1 
                         AND f.fo_value = 'N'
                         AND t1.case_number = f.case_number)
        AS last_no_final_order,    
    (SELECT c.entry_date FROM __temp__.ca_case_closed_events c
                         WHERE c.min_date = 1 
                         AND t1.case_number = c.case_number)
        AS first_close_date,
    (SELECT c.entry_date FROM __temp__.ca_case_closed_events c
                         WHERE c.max_date = 1 
                         AND t1.case_number = c.case_number) 
        AS last_close_date,                 
    (SELECT r.entry_date FROM __temp__.ca_case_reopened_events r
                         WHERE r.min_date = 1 
                         AND t1.case_number = r.case_number)
        AS first_reopen,
    (SELECT r.entry_date FROM __temp__.ca_case_reopened_events r
                         WHERE r.max_date = 1 
                         AND t1.case_number = r.case_number)
        AS last_reopen
FROM
  __temp__.ca_disp_events t1
""",
    
"ca_date_flags")

In [ ]:
pydb.create_temp_table(
f"""
Select
  *,
  CASE WHEN last_no_final_order IS NOT NULL
       AND last_final_order IS NULL
       AND last_case_close IS NOT NULL 
       AND last_no_final_order > last_reopen
     THEN 1 ELSE 0 END
    AS no_final_order_close,
  CASE WHEN last_final_order IS NOT NULL 
       AND last_final_order > last_no_final_order
       AND last_final_order > last_reopen
      THEN 1 ELSE 0 END
    AS final_order_close
    
   

In [89]:
pydb.create_temp_table(
f"""
Select
  *,
  CASE WHEN last_final_order >= last_no_final_order AND last_final_order >= last_close_date AND last_final_order >= last_reopen
       THEN 1 ELSE 0 END 
    AS last_date_is_final_order,
  CASE WHEN last_close_date >= last_final_order AND last_close_date >= last_no_final_order AND last_close_date >= last_reopen
       THEN 1 ELSE 0 END 
    AS last_date_is_case_closed,  
  CASE WHEN (last_no_final_order >= last_close_date AND last_no_final_order >= last_final_order)
        OR (last_reopen >= last_close_date AND last_no_final_order >= last_final_order)
       THEN 1 ELSE 0 END
    AS case_not_closed
FROM
  __temp__.ca_date_flags

""",
    
"ca_last_date")
  

In [90]:
pydb.read_sql_query ("select * from __temp__.ca_last_date")

,case_number,first_final_order,last_final_order,first_no_final_order,last_no_final_order,first_close_date,last_close_date,first_reopen,last_reopen,last_date_is_final_order,last_date_is_case_closed,case_not_closed
0,BM13P08280,2016-11-10 13:15:35,2016-11-10 13:15:35,2013-03-11 10:23:15,2016-05-19 15:28:11,2014-02-22 12:15:33,2016-11-10 13:16:22,2015-06-19 14:13:12,2016-02-20 09:02:59,0,1,0
1,EQ11P00021,2011-09-29 10:01:37,2011-09-29 10:01:37,NaT,NaT,2011-09-29 10:04:57,2011-09-29 10:04:57,NaT,NaT,0,0,0
2,PE01P00730,NaT,NaT,2001-09-24 13:56:28,2001-09-24 13:56:28,2005-02-16 08:40:12,2005-02-16 08:40:12,NaT,NaT,0,0,0
3,BS03P00020,2003-08-14 12:56:27,2003-11-11 14:10:27,2003-03-05 16:47:42,2004-08-19 15:43:39,2003-08-14 13:04:40,2004-09-02 10:35:56,2003-08-14 13:06:08,2003-11-17 10:10:20,0,1,0
4,EG14P00022,2014-04-04 13:37:00,2014-04-04 13:37:00,NaT,NaT,2014-04-04 13:37:51,2014-04-04 13:37:51,NaT,NaT,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2517738,KH00P05614,NaT,NaT,2001-03-02 14:32:16,2004-09-22 16:02:47,2004-11-01 10:39:12,2004-11-01 10:39:12,NaT,NaT,0,0,0
2517739,LN20P00116,2021-05-26 09:17:43,2021-05-26 09:17:43,2020-06-29 13:31:21,2020-06-29 13:31:21,2021-05-26 09:18:27,2021-05-26 09:18:27,NaT,NaT,0,0,0
2517740,KE09P00222,2010-05-07 15:22:12,2011-03-18 14:56:10,2009-11-26 11:23:15,2010-09-23 15:47:11,2010-05-07 15:23:08,2011-03-18 14:56:54,NaT,NaT,0,0,0
2517741,UV12P00128,2012-09-11 13:28:31,2012-09-11 13:28:31,NaT,NaT,2012-09-11 13:36:49,2012-09-11 13:36:49,NaT,NaT,0,0,0


In [116]:
s1 = f"""
select 
  last_final_order,
  last_no_final_order,
  last_close_date,
  last_reopen,
  CASE WHEN last_final_order = greatest (coalesce(last_final_order,last_no_final_order,last_close_date,last_reopen))
    THEN 1 ELSE 0 END 
   AS last_date_is_final_order
FROM
  __temp__.ca_date_flags
  limit 20
"""


In [117]:
simple_results = pydb.read_sql_query(s1)
simple_results

,last_final_order,last_no_final_order,last_close_date,last_reopen,last_date_is_final_order
0,2005-01-27 17:15:56,2005-01-06 14:44:20,2005-01-27 17:25:38,NaT,1
1,2009-12-02 11:25:49,2009-07-28 14:23:37,2009-12-02 11:26:29,NaT,1
2,2018-09-26 14:26:30,2018-09-11 11:17:16,2018-09-26 14:28:17,NaT,1
3,NaT,2016-04-22 15:53:21,2017-02-01 17:01:15,NaT,0
4,2014-06-30 08:39:31,NaT,2014-06-30 08:43:28,NaT,1
5,2021-02-17 12:00:17,NaT,2021-02-17 12:00:54,NaT,1
6,2016-08-25 16:43:38,2016-07-14 12:59:19,2016-08-25 16:44:46,NaT,1
7,2019-06-24 10:43:23,NaT,2019-06-24 10:43:51,2019-06-24 10:43:12,1
8,NaT,2012-09-12 12:29:44,2014-07-29 12:37:31,NaT,0
9,2020-12-03 06:32:33,2020-02-19 11:05:30,2020-12-03 06:33:55,NaT,1


In [101]:
s1 =f"""
select 
(SELECT MAX(LastUpdateDate) 
  FROM (VALUES (last_final_order),(last_no_final_order),(last_final_order)) AS lastDate(LastUpdateDate)) 
   AS LastUpdateDate
FROM
  __temp__.ca_date_flags
"""
